In [1]:
from transformers import AutoTokenizer
import pandas as pd

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
df = pd.read_csv('../data/emrqa_all_data.csv')
df.head()

,question,context,answer,answer_start,evidence
0,What is the patient's current dose does the pa...,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...
1,What is the current dose of valproic acid,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...
2,How much valproic acid does the patient take p...,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...
3,What is her current dose of valproic acid,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...
4,What is the current dose of the patient's valp...,RECORD #91996\n150823816 | DMC | 60956989 | | ...,250 mg,7,throughout this pregnancy. She has been taking...


In [37]:
test = df.sample(5).reset_index(drop=True)
test

,question,context,answer,answer_start,evidence
0,What is the patient's current dose does the pa...,RECORD #852130\n844726908 | IHC | 54526162 | |...,900 mg,2,10. Gemfibrozil 900 mg p.o. twice daily.\n
1,What treatments for leak from relaford pouch h...,Admission Date :\n2014-02-21\nDischarge Date :...,repair,8,She recieved further resection of remaining co...
2,Does he have hyperlipidemia,Record date: 2065-04-28 ME...,Hypercholesterolemia,5168,"PAST MEDICAL HISTORY: Hypercholesterolemia, g..."
3,Has the patient ever screened postive on incom...,648746074\nCTMC\n84164942\n960919\n9/7/1992 12...,ekg,0,EKG showed sinus tachycardia and incomplete ri...
4,when was the patient's last hyperlipidemia,"Record date: 2079-01-14PICH 989 James Street, ...",2081-10-07,21104,Record date: 2081-10-07\n\n Ms. Greenwood is a...


In [38]:
row = test.iloc[0]
row

question        What is the patient's current dose does the pa...
context         RECORD #852130\n844726908 | IHC | 54526162 | |...
answer                                                     900 mg
answer_start                                                    2
evidence               10. Gemfibrozil 900 mg p.o. twice daily.\n
Name: 0, dtype: object

In [39]:
inputs = tokenizer(test['question'].tolist(), test['evidence'].tolist(), truncation='only_second',
                   stride=50, return_overflowing_tokens=True, return_offsets_mapping=True)
print(f"The 5 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 5 examples gave 5 features.
Here is where each comes from: [0, 1, 2, 3, 4].


In [40]:
test['answer']

0                  900 mg
1                  repair
2    Hypercholesterolemia
3                     ekg
4              2081-10-07
Name: answer, dtype: object

In [44]:
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = test['answer'][sample_idx]
    start_char = test["answer_start"][sample_idx]
    end_char = test["answer_start"][sample_idx] + len(answer)
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([22, 17, 0, 15, 0], [24, 19, 0, 16, 0])

In [45]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = test['answer'][idx]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: 900 mg, labels give: . gemfi
